In [ ]:

from langchain.document_loaders import TextLoader
from pypdf import PdfReader
from langchain import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory

import functools
import time
from concurrent.futures import ThreadPoolExecutor
from typing import Generator, List, Tuple
from typing import Optional, TypeVar
import math
from typing import Any
from pathlib import Path
import os
import json
import numpy as np
from tqdm.auto import tqdm
from vertexai.preview.language_models import TextEmbeddingModel
# Single File read block
# Specify the directory you want to read

documents_1 = ''
documents_directory = '/home/jupyter/rbi-bot/rbi-docs'
splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20
)

def split_documents(splitter, documents_1):
    # Split the documents using the provided splitter
    split_1 = splitter.split_text(documents_1)
    # Create documents from the split chunks
    split_1 = splitter.create_documents(split_1)
    return split_1

vector_db = None
filepath = '/home/jupyter/rbi-bot/rbi-docs/01MC01042024E0D6B768164C41678A616F743BF7426B 2.pdf'
with open(filepath, 'r') as file:
    # Read the contents of the file
    reader = PdfReader(filepath)
    for page in reader.pages:
        documents_1 += page.extract_text()
        print(f"generated documents")
    splits_1 = split_documents(splitter,documents_1)
    print(f"split documents")
    #extension_db = FAISS.from_documents(splits_1, model)
    #extension_db.save_local(vector_db_directory) 
    #print(splits_1)

In [ ]:
!pip install openai

In [ ]:
#OpenAI embedding
from openai import OpenAI
client = OpenAI(api_key="sk-rbi-bot-service-account-xJqcUMGqeOw7uLU86lOFT3BlbkFJdn7MkxPGycAAL8bzlkzx")


def encode_texts_to_openAI_embeddings(sentences: List[str]) -> List[Optional[List[float]]]:
    print("batch size ", len(sentences))
    try:
        response = client.embeddings.create(input = sentences, model="text-embedding-3-small").data
        embeddings= [obj[embedding] for obj in response]
        return [embedding.values for embedding in embeddings]
    except Exception as e:
        print("exception", e)
        return [None for _ in range(len(sentences))]
    
embeddings = encode_texts_to_openAI_embeddings(["this is a text1", "this is a text 2"])
print(embeddings)